In [3]:
%pip install experta

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --upgrade frozendict

  Attempting uninstall: frozendict
    Found existing installation: frozendict 1.2
    Uninstalling frozendict-1.2:
      Successfully uninstalled frozendict-1.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
experta 1.9.4 requires frozendict==1.2, but you have frozendict 2.4.6 which is incompatible.


#### Decision Tree Modelling (Working adults)

##### Inputs:

- Age
- Planning to buy a home?
- Still repaying home loans?
- Have kids?
- Supporting aged parents?
- Monthly take-home pay
  - IF don't know THEN ask for gross salary AND whether there is CPF contribution
- Estimated current expenses on needs / wants / insurance / investment
- Estimated current emergency funds

##### Recommended Actions:

- Needs Analysis:
  - Transportation
  - Food
  - Utility bill
  - Home loan payment / Rent expense
  - Insurance expense
  - Medical expense
  - Others
  - IF transport expenditure > $150 AND (needs expenditure over budget OR too little savings): RECOMMEND taking public transport instead
  - IF utility bill > 300: RECOMMEND electricity/water saving methods or switching utilities provider 
  - IF insurance expenditure > 15% of take-home salary: REDUCE insurance expenditure
  - IF needs expenditure > 50% of take-home salary: REDUCE spending on needs
    - IF food expense > 15% of take-home salary: REDUCE spending on food
    - IF housing expense > 35% of take-home salary: RECOMMEND finding alternatives
- Wants Analysis:
  - IF wants expenditure > 30% of take-home salary: REDUCE spending on wants
- Savings Analysis:
  - Investments
  - Savings in bank
  - IF investment expenditure < 10% of take-home salary: 
    - IF age < 65: INVEST in long-term investment plans such as ETFs for retirement
    - IF have kids: INVEST in short-term investment plans for kids' tertiary education
    - IF planning to buy a home: INVEST in short-term investment plans for home loan repayment
  - IF monthly savings < 20% of take-home salary: INCREASE savings by REDUCING spending on wants or needs
- Emergency Funds Analysis:
  - IF emergency funds < 3 months of current expenses: INCREASE savings
  - ELSE IF emergency funds < 6 months of current expenses (needs + wants): RECOMMENDED to increase savings


In [11]:
# EXAMPLE USAGE OF EXPERTA

from experta import KnowledgeEngine, Rule, Fact, MATCH, TEST

# Define a Fact subclass for budget-related data
class BudgetFact(Fact):
    # You can define expected fields here if you want to enforce a schema.
    needs_expenditure = float
    take_home_income = float

# Create a KnowledgeEngine subclass with rules
class BudgetAdvisor(KnowledgeEngine):

    @Rule(BudgetFact(needs_expenditure=MATCH.needs, take_home_income=MATCH.income),
          TEST(lambda needs, income: needs > 0.5 * income))
    def high_needs_expenditure(self):
        print("Your needs expenditure is over 50% of your take-home income. Consider reducing spending on essential needs.")

    @Rule(BudgetFact(needs_expenditure=MATCH.needs, take_home_income=MATCH.income),
          TEST(lambda needs, income: needs <= 0.5 * income))
    def acceptable_needs_expenditure(self):
        print("Your needs expenditure is within an acceptable range.")

# Using the engine
engine = BudgetAdvisor()
engine.reset()  # Prepare the engine for new facts
# Declare a fact: For example, needs expenditure is 3000 SGD and take-home income is 5000 SGD.
engine.declare(BudgetFact(needs_expenditure=3000, take_home_income=5000))
engine.run()  # Process rules and execute matching ones

Your needs expenditure is over 50% of your take-home income. Consider reducing spending on essential needs.


In [1]:
from experta import KnowledgeEngine, Fact, Rule, MATCH, TEST

class BudgetInfo(Fact):
    """Holds the user's budget-related information."""
    # Basic personal details
    age = int
    planning_to_buy_home = bool
    repaying_home_loans = bool
    have_kids = bool
    supporting_aged_parents = bool
    monthly_take_home = float

    # Expenses and savings details
    transport_expenditure = float
    needs_expenditure = float
    wants_expenditure = float
    insurance_expenditure = float
    food_expenditure = float
    housing_expenditure = float
    utility_bill = float
    investment_expenditure = float
    monthly_savings = float
    emergency_funds = float

class BudgetAdvisor(KnowledgeEngine):
    # ---------------- Needs Analysis Rules ----------------
    @Rule(BudgetInfo(transport_expenditure=MATCH.transport,
                     needs_expenditure=MATCH.needs,
                     monthly_take_home=MATCH.take_home,
                     monthly_savings=MATCH.savings),
          TEST(lambda transport, needs, take_home, savings: 
               transport > 150 and (needs > 0.5 * take_home or savings < 0.2 * take_home)))
    def rule_public_transport(self, transport, needs, take_home, savings):
        print("Your transport spendings are above average. Consider taking public transport instead of using private options.")

    @Rule(BudgetInfo(utility_bill=MATCH.utility),
          TEST(lambda utility: utility > 200))
    def rule_switch_utilities(self, utility):
        print("To conserve utilities and reduce costs, consider using energy-efficient appliances, " \
        "turning off lights when leaving a room, and taking shorter showers. " \
        "You may also consider switching your utilities provider to reduce bills.")

    @Rule(BudgetInfo(insurance_expenditure=MATCH.insurance, monthly_take_home=MATCH.take_home),
          TEST(lambda insurance, take_home: insurance > 0.15 * take_home))
    def rule_reduce_insurance(self, insurance, take_home):
        print("Your spending on insurance is too high! " \
        "The recommended spending on insurance is maximum 15% of your take home-salary.")

    @Rule(BudgetInfo(needs_expenditure=MATCH.needs, monthly_take_home=MATCH.take_home),
          TEST(lambda needs, take_home: needs > 0.5 * take_home))
    def rule_reduce_needs(self, needs, take_home):
        print("Your spending on necessities are too high! " \
        "The recommended spending on needs is maximum 50% of your take-home salary.")

    @Rule(BudgetInfo(food_expenditure=MATCH.food, monthly_take_home=MATCH.take_home),
          TEST(lambda food, take_home: food > 0.15 * take_home))
    def rule_reduce_food(self, food, take_home):
        print("Your spending on food are too high! " \
        "The recommended spending on food is maximum 15% of your take-home salary.")

    @Rule(BudgetInfo(housing_expenditure=MATCH.housing, monthly_take_home=MATCH.take_home),
          TEST(lambda housing, take_home: housing > 0.35 * take_home))
    def rule_housing_alternatives(self, housing, take_home):
        print("You may want to consider looking for alternative housing options " \
        "as housing costs are over 35% of your take-home salary.")

    # ---------------- Wants Analysis Rule ----------------
    @Rule(BudgetInfo(wants_expenditure=MATCH.wants, monthly_take_home=MATCH.take_home),
          TEST(lambda wants, take_home: wants > 0.3 * take_home))
    def rule_reduce_wants(self, wants, take_home):
        print("Your spending on wants are too high! " \
        "The recommended spending on wants is maximum 30% of your take-home salary.")

    # ---------------- Savings Analysis Rules ----------------
    @Rule(BudgetInfo(investment_expenditure=MATCH.invest,
                     monthly_take_home=MATCH.take_home,
                     age=MATCH.age,
                     have_kids=MATCH.kids,
                     planning_to_buy_home=MATCH.buy_home),
          TEST(lambda invest, take_home: invest < 0.1 * take_home))
    def rule_investment_recommendations(self, age, kids, buy_home):
        if age < 65:
            print("Consider investing in long-term plans (e.g. ETFs) for retirement.")
        if kids:
            print("Consider short-term investment plans for your kids' tertiary education.")
        if buy_home:
            print("Consider short-term investment plans to help with home loan repayment.")

    @Rule(BudgetInfo(monthly_savings=MATCH.savings, monthly_take_home=MATCH.take_home),
          TEST(lambda savings, take_home: savings < 0.2 * take_home))
    def rule_increase_savings(self, savings, take_home):
        print("Your savings are too low! You are recommended to save 20% of your take-home salary. " \
        "You can increase your monthly savings by reducing spending on needs or wants.")

    # ---------------- Emergency Funds Analysis Rules ----------------
    @Rule(BudgetInfo(emergency_funds=MATCH.funds, needs_expenditure=MATCH.needs, wants_expenditure=MATCH.wants),
          TEST(lambda funds, needs, wants: funds < 3 * (needs + wants)))
    def rule_emergency_funds_low(self, funds, needs, wants):
        print("Financial experts recommend maintaining an emergency fund covering 3-6 months of your monthly expenses. " \
        "Currently, your savings fall short of the 3-month minimum. " \
        "To avoid financial stress during unexpected events, please consider increasing your savings as soon as possible.")

    @Rule(BudgetInfo(emergency_funds=MATCH.funds, needs_expenditure=MATCH.needs, wants_expenditure=MATCH.wants),
          TEST(lambda funds, needs, wants: funds > 3 * (needs + wants) and funds < 6 * (needs + wants)))
    def rule_emergency_funds_medium(self, funds, needs, wants):
        print("Financial experts recommend maintaining an emergency fund covering 3-6 months of your monthly expenses. " \
        "Build up your emergency funds to cover at least 6 months of your current expenses.")

if __name__ == "__main__":
    engine = BudgetAdvisor()
    engine.reset()

    # Example inputs – these values could be gathered interactively or via a user interface.
    engine.declare(BudgetInfo(
        age=30,
        planning_to_buy_home=True,
        repaying_home_loans=False,
        have_kids=True,
        supporting_aged_parents=False,
        monthly_take_home=4000,
        transport_expenditure=200,
        needs_expenditure=2500,
        wants_expenditure=1500,
        insurance_expenditure=700,
        food_expenditure=700,
        housing_expenditure=1000,
        utility_bill=250,
        investment_expenditure=300,
        monthly_savings=800,
        emergency_funds=8000
    ))

    engine.run()


To conserve utilities and reduce costs, consider using energy-efficient appliances, turning off lights when leaving a room, and taking shorter showers. You may also consider switching your utilities provider to reduce bills.
Financial experts recommend maintaining an emergency fund covering 3-6 months of your monthly expenses. Currently, your savings fall short of the 3-month minimum. To avoid financial stress during unexpected events, please consider increasing your savings as soon as possible.
Your spending on insurance is too high! The recommended spending on insurance is maximum 15% of your take home-salary.
Your spending on food are too high! The recommended spending on food is maximum 15% of your take-home salary.
Your spending on wants are too high! The recommended spending on wants is maximum 30% of your take-home salary.
Consider investing in long-term plans (e.g. ETFs) for retirement.
Consider short-term investment plans for your kids' tertiary education.
Consider short-term 